# Integration of lakeFS with Glue Catalog and Athena

[📚 Docs](https://docs.lakefs.io/integrations/glue_metastore.html)

## Use Case: Isolated Dev/Test Environments

## Config

### Glue session configuration

In [ ]:
%stop_session
%session_id_prefix 'lakefs-glue-exporter-demo'
%idle_timeout 120
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

%additional_python_modules 'lakefs'

### lakeFS endpoint and credentials

In [ ]:
lakefsEndPoint = '<lakeFS Endpoint URL>' # e.g. 'https://username.aws_region_name.lakefscloud.io' 
lakefsAccessKey = '<lakeFS Access Key>'
lakefsSecretKey = '<lakeFS Secret Key>'

### AWS credentials

In [ ]:
awsAccessKeyId = '<YOUR_AwsAccessKey_HERE>'
awsSecretAccessKey = '<YOUR_AwsSecretKey_HERE>'
awsRegion = '<YOUR_AwsRegion_HERE>'

### Object Storage

In [ ]:
storageNamespace = 's3://<Bucket Name>' # e.g. 's3://bucket'

### Glue Catalog Information

In [ ]:
glueDatabaseName = "glue_exporter_demo" # This notebook will create this database
glueCatalogId = "<Glue Catalog ID or AWS Account ID>"
BucketNameForGlueCatalog = '<Bucket Name to store Glue Catalog>'

### Athena Information

In [ ]:
BucketNameForAthenaOutput = '<Bucket Name to store Athena Output>'
FolderNameForAthenaOutput = 'lakefs-glue-exporter-demo'

### Install and configure lakectl (lakeFS command-line tool): https://docs.lakefs.io/reference/cli.html

---

## Setup

**(you shouldn't need to change anything in this section, just run it)**

In [ ]:
repo_name = "glue-exporter-demo"

### Versioning Information

In [ ]:
mainBranch = "main"
glueTestBranch = "glue_test_branch"
customersTable = "customers"

### Import libraries

In [ ]:
import os
import lakefs
import yaml
from pyspark.sql.types import ByteType, IntegerType, LongType, StringType, StructType, StructField
from pyspark.sql.functions import *

### Create lakeFSClient

In [ ]:
os.environ["LAKECTL_SERVER_ENDPOINT_URL"] = lakefsEndPoint
os.environ["LAKECTL_CREDENTIALS_ACCESS_KEY_ID"] = lakefsAccessKey
os.environ["LAKECTL_CREDENTIALS_SECRET_ACCESS_KEY"] = lakefsSecretKey

#### Verify lakeFS credentials by getting lakeFS version

In [ ]:
print("Verifying lakeFS credentials…")
try:
    v=lakefs.client.Client().version
except:
    print("🛑 failed to get lakeFS version")
else:
    print(f"…✅lakeFS credentials verified\n\nℹ️lakeFS version {v}")

### Define lakeFS Repository

In [17]:
repo = lakefs.Repository(repo_name).create(storage_namespace=f"{storageNamespace}/{repo_name}", default_branch=mainBranch, exist_ok=True)
branchMain = repo.branch(mainBranch)
print(repo)

{'id': 'amit-glue-exporter-demo3', 'creation_date': 1722381378, 'default_branch': 'main', 'storage_namespace': 's3://treeverse-ort-simulation-bucket/amit/amit-glue-exporter-demo3'}


### Set up Spark

In [18]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key",lakefsSecretKey)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint",lakefsEndPoint)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key",lakefsAccessKey)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access","true")

### Create Glue Database

In [20]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {glueDatabaseName} LOCATION 's3://{BucketNameForGlueCatalog}/'").show()

++
||
++
++


### Define some helper functions

In [21]:
def print_diff_refs(diff_refs):
    results = map(
        lambda n:[n.path,n.path_type,n.size_bytes,n.type],
        diff_refs.results)
    return results

def print_commit(log):
    from datetime import datetime
    from pprint import pprint

    print('Message:', log.message)
    print('ID:', log.id)
    print('Committer:', log.committer)
    print('Creation Date:', datetime.utcfromtimestamp(log.creation_date).strftime('%Y-%m-%d %H:%M:%S'))
    print('Parents:', log.parents)
    print('Metadata:')
    pprint(log.metadata)

In [22]:
import boto3
import pandas as pd

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
athena_client = boto3.client(service_name='athena', region_name=awsRegion)

def run_query(client, query):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={ 'Database': glueDatabaseName },
        ResultConfiguration={ 'OutputLocation': 's3://{}/{}/'.format(BucketNameForAthenaOutput, FolderNameForAthenaOutput) },
    )
    return response

def validate_query(client, query_id):
    resp = ["FAILED", "SUCCEEDED", "CANCELLED"]
    response = client.get_query_execution(QueryExecutionId=query_id)
    # wait until query finishes
    while response["QueryExecution"]["Status"]["State"] not in resp:
        response = client.get_query_execution(QueryExecutionId=query_id)

    return response["QueryExecution"]["Status"]["State"]

def execute_athena_query(query):
    print('start query: {}\n'.format(query))
    qe = run_query(athena_client, query)
    qstate = validate_query(athena_client, qe["QueryExecutionId"])
    print('query state: {}\n'.format(qstate))

    file_name = "{}/{}.csv".format(FolderNameForAthenaOutput, qe["QueryExecutionId"])
    obj = s3_client.get_object(Bucket=BucketNameForAthenaOutput, Key=file_name)
    return pd.read_csv(obj['Body'])

### Define CUSTOMER.csv data file schema

In [23]:
customersSchema = StructType([
  StructField("Customer_ID", IntegerType(), False),
  StructField("Country", StringType(), False),
  StructField("Gender", StringType(), False),
  StructField("Personal_ID", IntegerType(), True),
  StructField("Customer_Name", StringType(), False),
  StructField("Customer_FirstName", StringType(), False),
  StructField("Customer_LastName", StringType(), False),
  StructField("Birth_Date", StringType(), False),
  StructField("Customer_Address", StringType(), False),
  StructField("Street_ID", LongType(), False),
  StructField("Street_Number", IntegerType(), False),
  StructField("Customer_Type_ID", IntegerType(), False)
])

### Let’s define the table descriptor and upload it to lakeFS

In [24]:
table_descriptor = {
    'name': customersTable,
    'type': 'hive',
    'path': f'tables/{customersTable}',
    'schema': {
        'type': 'struct',
        # all the columns spec
        'fields': [
            {
                'name': 'Customer_ID',
                'type': 'integer',
                'nullable': False,
                'metadata': {
                    'comment': 'Customer ID'
                }
            },
            {
                'name': 'Country',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Gender',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Personal_ID',
                'type': 'integer',
                'nullable': True,
                'metadata': {}
            },
            {
                'name': 'Customer_Name',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Customer_FirstName',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Customer_LastName',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Birth_Date',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Customer_Address',
                'type': 'string',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Street_ID',
                'type': 'bigint',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Street_Number',
                'type': 'integer',
                'nullable': False,
                'metadata': {}
            },
            {
                'name': 'Customer_Type_ID',
                'type': 'integer',
                'nullable': False,
                'metadata': {}
            }
        ]
    }
}

# Write table descriptor to lakeFS
with branchMain.object(path=f'_lakefs_tables/{customersTable}.yaml').writer() as out:
    yaml.safe_dump(table_descriptor, out)

### Upload the Glue Catalog exporter script to lakeFS

In [25]:
luaScriptName = "scripts/glue_export.lua"

lua_script = """

local aws = require("aws")
local symlink_exporter = require("lakefs/catalogexport/symlink_exporter")
local glue_exporter = require("lakefs/catalogexport/glue_exporter")
-- settings 
local access_key = args.aws.aws_access_key_id
local secret_key = args.aws.aws_secret_access_key
local region = args.aws.aws_region
local table_path = args.table_source -- table descriptor 
local db = args.catalog.db_name -- glue db
local table_input = args.catalog.table_input -- table input (AWS input spec) for Glue
-- export symlinks 
local s3 = aws.s3_client(access_key, secret_key, region)
local result = symlink_exporter.export_s3(s3, table_path, action, {debug=true})
-- register glue table
local glue = aws.glue_client(access_key, secret_key, region)
local res = glue_exporter.export_glue(glue, db, table_path, table_input, action, {debug=true})

"""

branchMain.object(path=luaScriptName).upload(data=lua_script, mode='wb')

WriteableObject(repository="amit-glue-exporter-demo3", reference="main", path="scripts/glue_export.lua")


### Define an action configuration that will run the above script after a commit/merge is completed over the main or glueTestBranch branches or after glueTestBranch branch is created.
### Upload it to lakeFS.

In [26]:
hook_definition = {
    'name': 'Glue Exporter',
    'on': {
        'post-commit': {
            'branches': [mainBranch, glueTestBranch+'*']
        },
        'post-create-branch': {
            'branches': [glueTestBranch+'*']
        }
    },
    'hooks': [
        {
            'id': 'Glue-Registration',
            'type': 'lua',
            'properties': {
                'script_path': luaScriptName,
                'args': {
                    'aws': {
                        'aws_access_key_id': awsAccessKeyId,
                        'aws_secret_access_key': awsSecretAccessKey,
                        'aws_region': awsRegion
                    },
                    'table_source': f'_lakefs_tables/{customersTable}.yaml',
                    'catalog': {
                        'db_name': glueDatabaseName,
                        'table_input': {
                            'StorageDescriptor': {
                                'InputFormat': 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat',
                                'OutputFormat': 'org.apache.hadoop.hive.ql.io.IgnoreKeyTextOutputFormat',
                                'SerdeInfo': {
                                   'SerializationLibrary': 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
                                }
                            },
                            'Parameters': {
                                'classification': 'parquet',
                                'EXTERNAL': 'TRUE',
                                'parquet.compression': 'SNAPPY'
                            }
                        }
                    }
                }
            }
        }
    ]
}

with branchMain.object(path='_lakefs_actions/glue_exporter_action.yaml').writer() as out:
    yaml.safe_dump(hook_definition, out)

---

# Main demo starts here 🚦 👇🏻

For this demo - we'll be utilizing a dataset - [Orion Star - Sports and outdoors RDBMS dataset](https://www.kaggle.com/datasets/chethanp11/orion-star-sports-and-outdoors-rdbms-dataset) from [Kaggle](https://www.kaggle.com/).

## Run following command on your computer to clone lakeFS samples repo along with sample data used by this notebook:

### git clone https://github.com/treeverse/lakeFS-samples.git

## Print the command and run it on your computer to upload sample data to lakeFS repository

In [27]:
print(f"cd lakeFS-samples && lakectl fs upload -s ./data/OrionStar lakefs://{repo.id}/main/data/OrionStar/ --recursive && lakectl commit lakefs://{repo.id}/main -m 'Uploaded sample data'")

cd lakeFS-samples && lakectl fs upload -s ./data/OrionStar lakefs://amit-glue-exporter-demo3/main/data/OrionStar/ --recursive && lakectl commit lakefs://amit-glue-exporter-demo3/main -m 'Uploaded sample data'


## Create Customers table in the main branch (using [CUSTOMER.csv](./data/samples/OrionStar/CUSTOMER.csv) file)

In [28]:
customersTablePath = f"s3a://{repo.id}/{mainBranch}/tables/{customersTable}"
print(customersTablePath)

s3a://amit-glue-exporter-demo3/main/tables/customers


#### Read CSV file and create Hive table

In [29]:
df = spark.read.csv(f"s3a://{repo.id}/{mainBranch}/data/OrionStar/CUSTOMER.csv",header=True,schema=customersSchema)
df.write.parquet(customersTablePath)
df.show(10)

+-----------+-------+------+-----------+-----------------+------------------+-----------------+----------+--------------------+----------+-------------+----------------+
|Customer_ID|Country|Gender|Personal_ID|    Customer_Name|Customer_FirstName|Customer_LastName|Birth_Date|    Customer_Address| Street_ID|Street_Number|Customer_Type_ID|
+-----------+-------+------+-----------+-----------------+------------------+-----------------+----------+--------------------+----------+-------------+----------------+
|          4|     US|     M|       null|    James Kvarniq|             James|          Kvarniq| 27JUN1974|      4382 Gralyn Rd|9260106519|         4382|            1020|
|          5|     US|     F|       null|Sandrina Stephano|          Sandrina|         Stephano| 09JUL1979|    6468 Cog Hill Ct|9260114570|         6468|            2020|
|          9|     DE|     F|       null|   Cornelia Krahl|          Cornelia|            Krahl| 27FEB1974|   Kallstadterstr. 9|3940106659|            

## Commit changes and attach some metadata

In [30]:
ref = branchMain.commit(
    message="Added configuration files and customers tables!",
    metadata={'using': 'python_api'})
print_commit(ref.get_commit())

Message: Added configuration files and customers tables!
ID: 45693acbc23e5776f44ca961db846dcb140b81421eb18016df056a7db980909b
Committer: amit.kesarwani@treeverse.io
Creation Date: 2024-07-30 23:20:11
Parents: ['d1da2f078baae4868ced0c2d6d7dd2572c1701dac735ba170872eeb552f2736a']
Metadata:
{'using': 'python_api'}


## Execute Athena query to read the data

In [31]:
print(execute_athena_query(f'SELECT * FROM "{glueDatabaseName}"."{customersTable}_{repo_name}_{mainBranch}_{ref.get_commit().id[:6]}"'))

start query: SELECT * FROM "amit_glue_exporter_demo"."customers_amit-glue-exporter-demo3_main_45693a"

query state: SUCCEEDED

    customer_id country gender  ...   street_id street_number customer_type_id
0             4      US      M  ...  9260106519          4382             1020
1             5      US      F  ...  9260114570          6468             2020
2             9      DE      F  ...  3940106659             9             2020
3            10      US      F  ...  9260129395           425             1040
4            11      DE      F  ...  3940108592            15             1040
..          ...     ...    ...  ...         ...           ...              ...
72        70165      CA      F  ...  2600100006           873             1020
73        70187      CA      F  ...  2600100035          1835             1030
74        70201      CA      F  ...  2600100012           319             2010
75        70210      CA      M  ...  2600100015            40             1030
76  

# 🟢 ETL Job Starts

## Create a new branch to run your ETL job

In [32]:
branchGlueTest = repo.branch(glueTestBranch).create(source_reference=mainBranch)
print(f"{glueTestBranch} ref:", branchGlueTest.get_commit().id)

glue_test_branch ref: 45693acbc23e5776f44ca961db846dcb140b81421eb18016df056a7db980909b


## Execute Athena query to read the data from the new branch

In [33]:
print(execute_athena_query(f'SELECT * FROM "{glueDatabaseName}"."{customersTable}_{repo_name}_{glueTestBranch}_{branchGlueTest.get_commit().id[:6]}"'))

start query: SELECT * FROM "amit_glue_exporter_demo"."customers_amit-glue-exporter-demo3_glue_test_branch_45693a"

query state: SUCCEEDED

    customer_id country gender  ...   street_id street_number customer_type_id
0             4      US      M  ...  9260106519          4382             1020
1             5      US      F  ...  9260114570          6468             2020
2             9      DE      F  ...  3940106659             9             2020
3            10      US      F  ...  9260129395           425             1040
4            11      DE      F  ...  3940108592            15             1040
..          ...     ...    ...  ...         ...           ...              ...
72        70165      CA      F  ...  2600100006           873             1020
73        70187      CA      F  ...  2600100035          1835             1030
74        70201      CA      F  ...  2600100012           319             2010
75        70210      CA      M  ...  2600100015            40          

## More Questions?

###### Join the lakeFS Slack group - https://lakefs.io/slack